# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210205_122504_cgan_predict_0.8_m2', '20210209_1510…

In [ ]:
# result_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201213_173446_lambda0.1_good_models'

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210209_075831_lambda2.0


### Get validation data

In [11]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [12]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


## Plot Losses

In [13]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [14]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
62491,62491.0,19.0,0.125412,0.122259,0.247671,7.763586,160.005966,152.242386,-2.190622,155.126831,-1.961364,5.865645,-4.036070,-7.761815,0.144420
62492,62492.0,19.0,0.225140,0.099111,0.324251,5.029626,166.919006,161.889374,-2.729531,162.491684,-2.606127,2.629968,-6.278219,-5.005906,0.143939
62493,62493.0,19.0,0.079710,0.140343,0.220052,4.318383,168.290314,163.971924,-2.734797,161.854858,-2.701814,4.775238,-4.384341,-4.280250,0.143481
62494,62494.0,19.0,0.102162,0.224648,0.326810,5.261605,167.457626,162.196014,-2.795750,161.251770,-2.920872,5.279943,-3.329255,-5.246347,0.142167
62495,62495.0,19.0,0.140343,0.146696,0.287039,4.896984,169.342575,164.445587,-2.876992,174.103424,-2.570870,3.368156,-3.963730,-4.868117,0.144439
62496,62496.0,19.0,0.195260,0.161835,0.357095,4.133395,161.835052,157.701660,-2.894624,157.120255,-2.931280,3.249411,-4.817617,-4.073566,0.145952
62497,62497.0,19.0,0.140725,0.432027,0.572752,5.975332,166.198593,160.223267,-2.950042,161.476547,-2.551576,4.563378,-2.181591,-5.967602,0.144209
62498,62498.0,19.0,0.324014,0.120196,0.444210,3.200354,163.299866,160.099518,-2.863214,169.127640,-3.061589,1.901509,-4.987773,-3.090899,0.143444
62499,62499.0,19.0,0.124956,0.211952,0.336908,4.620792,158.101120,153.480331,-2.737945,159.289169,-2.875651,4.668356,-2.707228,-4.596022,0.143420
62500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.031616,-2.270388,NaN,NaN,NaN,NaN


In [15]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [16]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-2.954545166492462


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
57949,57949.0,18.0,0.159918,0.120515,0.280432,4.452913,165.027954,160.575043,-2.942075,162.808395,-3.129984,4.695415,-3.266768,-4.427402,0.130169
59878,59878.0,19.0,0.084244,0.238540,0.322784,5.698692,168.756470,163.057785,-3.037373,171.807190,-3.111753,4.708845,-2.804726,-5.692707,0.129576
61228,61228.0,19.0,0.897960,0.170494,1.068454,4.405140,158.224350,153.819214,-2.946219,159.823990,-3.102009,0.389696,-10.006269,-4.326868,0.131889
60144,60144.0,19.0,0.108392,0.186278,0.294670,7.653922,166.500168,158.846252,-3.076837,161.598297,-3.094000,5.760403,-3.334160,-7.652602,0.144676
57946,57946.0,18.0,0.046380,0.411710,0.458091,6.741944,161.128006,154.386063,-3.066014,168.055450,-3.090133,6.286459,-1.910878,-6.739133,0.131099
52985,52985.0,16.0,0.125205,0.078422,0.203627,6.133905,160.148148,154.014236,-3.033259,171.082886,-3.089963,4.254992,-5.019655,-6.126376,0.135786
62091,62091.0,19.0,0.171945,0.517832,0.689778,9.297056,168.778717,159.481659,-2.809314,177.470459,-3.089254,12.049929,-1.437172,-9.296888,0.130159
53396,53396.0,17.0,0.101494,0.132409,0.233903,4.412914,161.264633,156.851715,-3.036857,171.645081,-3.087489,4.291229,-4.294807,-4.378057,0.132161
57951,57951.0,18.0,0.140572,0.150043,0.290615,4.762429,161.608917,156.846481,-3.064131,153.613831,-3.078863,3.919338,-3.879446,-4.741371,0.128234
57955,57955.0,18.0,0.373458,0.118028,0.491486,2.800258,157.114243,154.313980,-3.075321,155.083939,-3.077277,1.600290,-6.810120,-2.634991,0.129454


In [17]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
57949,57949.0,18.0,0.159918,0.120515,0.280432,4.452913,165.027954,160.575043,-2.942075,162.808395,-3.129984,4.695415,-3.266768,-4.427402,0.130169
59878,59878.0,19.0,0.084244,0.238540,0.322784,5.698692,168.756470,163.057785,-3.037373,171.807190,-3.111753,4.708845,-2.804726,-5.692707,0.129576
61228,61228.0,19.0,0.897960,0.170494,1.068454,4.405140,158.224350,153.819214,-2.946219,159.823990,-3.102009,0.389696,-10.006269,-4.326868,0.131889
60144,60144.0,19.0,0.108392,0.186278,0.294670,7.653922,166.500168,158.846252,-3.076837,161.598297,-3.094000,5.760403,-3.334160,-7.652602,0.144676
57946,57946.0,18.0,0.046380,0.411710,0.458091,6.741944,161.128006,154.386063,-3.066014,168.055450,-3.090133,6.286459,-1.910878,-6.739133,0.131099
52985,52985.0,16.0,0.125205,0.078422,0.203627,6.133905,160.148148,154.014236,-3.033259,171.082886,-3.089963,4.254992,-5.019655,-6.126376,0.135786
62091,62091.0,19.0,0.171945,0.517832,0.689778,9.297056,168.778717,159.481659,-2.809314,177.470459,-3.089254,12.049929,-1.437172,-9.296888,0.130159
53396,53396.0,17.0,0.101494,0.132409,0.233903,4.412914,161.264633,156.851715,-3.036857,171.645081,-3.087489,4.291229,-4.294807,-4.378057,0.132161


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
55863,55863.0,17.0,0.066021,0.113937,0.179959,3.905745,160.233261,156.327515,-2.609807,147.781265,-2.518747,3.588452,-3.674779,-3.857825,0.146488
58020,58020.0,18.0,0.078548,0.152596,0.231144,5.130219,160.924561,155.794342,-2.671506,148.063873,-2.560497,5.069186,-3.502088,-5.117370,0.142075
52852,52852.0,16.0,0.122303,0.167199,0.289501,5.531900,155.836945,150.305038,-2.161132,148.113113,-2.197560,4.592605,-2.915690,-5.523962,0.135021
54744,54744.0,17.0,0.142365,0.112125,0.254490,5.437388,162.207642,156.770248,-2.483590,148.383377,-2.310849,3.010317,-5.833077,-5.420465,0.135225
53232,53232.0,17.0,0.187171,0.076068,0.263239,3.565162,157.415405,153.850250,-2.396410,148.434326,-2.374063,3.417173,-4.613925,-3.479200,0.131963
47180,47180.0,15.0,0.160548,0.128569,0.289116,4.079823,157.816650,153.736832,-2.810391,148.451172,-2.449859,3.163471,-5.792557,-4.009386,0.134294
52945,52945.0,16.0,0.145926,0.151877,0.297803,4.756117,166.137390,161.381271,-2.244182,148.541138,-2.218476,4.216842,-3.725899,-4.735287,0.132757
18325,18325.0,5.0,0.150669,0.099812,0.250481,4.364303,162.439407,158.075104,-1.502307,148.849854,-1.326953,4.168330,-4.387516,-4.329209,0.131531


## Read stored chi-squares for images

In [18]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [19]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [20]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [21]:
df_sliced=w.result
# df_sliced

In [22]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.037261712977445086, 'chi_spec1': 0.7115329996639242}
[0, 19]
{'chi_1': 0.1280929222993139, 'chi_spec1': 2.6275092295675364}
[0, 19]
{'chi_1': 0.037261712977445086, 'chi_spec1': 0.7115329996639242}
[array([39400, 50240, 58920, 56800]), array([26670, 15360]), array([53700, 55580])]
[15360 26670 39400 50240 53700 55580 56800 58920]


In [23]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [24]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(4, 15360), (8, 26670), (12, 39400), (16, 50240), (17, 53700), (17, 55580), (18, 56800), (18, 58920)]


In [25]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1535,4-15360,0.034308,1.172129,34.160445,0.017321,0.016145,0.000842,0.008596,15.593187,48.956244,4,15360
2666,8-26670,0.036176,0.810891,33.005505,0.020055,0.015288,0.000833,0.007470,17.490316,62.694544,8,26670
3939,12-39400,0.004118,0.160811,29.849727,0.000142,0.002396,0.001580,0.000026,14.009750,12.199097,12,39400
5023,16-50240,0.004472,0.078242,30.603662,0.000314,0.002687,0.001472,0.000030,13.755073,4.698342,16,50240
5369,17-53700,0.012779,0.038212,29.113524,0.002226,0.007243,0.003311,0.000200,14.764255,1.943800,17,53700
5557,17-55580,0.021039,0.047131,28.675669,0.006272,0.012020,0.002748,0.001403,14.003546,2.656229,17,55580
5679,18-56800,0.005464,0.296388,31.110349,0.000716,0.002195,0.002553,0.000184,13.766141,18.778109,18,56800
5891,18-58920,0.005340,0.620769,29.458043,0.001594,0.002366,0.001380,0.000899,13.370304,38.108521,18,58920


### Plots 2: stored data

In [26]:
def f_plot_hist_spec(df,dict_bkg,plot_type):
    
    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
    
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [27]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('4-15360', '8-26670', '12-39400', '16…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))


(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …